<a href="https://colab.research.google.com/github/leihui6/Robotics-MasterCourse/blob/main/Assignment_for_Robotics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Global Setting of Assignment

In [43]:
import numpy as np
from math import sqrt
#from numpy.linalg import inv, norm
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

def bmatrix(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

# The transformation from @F1 to @F2
def calculate_R_transformation(F1,F2):
    assert len(F1) == 3 and len(F2) == 3
    T = np.zeros((3,3))
    for F1_index,F1_item in enumerate(F1):
        for F2_index,F2_item in enumerate(F2):
            T[F2_index, F1_index] = np.dot(F1_item,F2_item.transpose())
    return T

# Combine rotation and translation transfomation
def add_T_transformation(R_T,T_T):
    assert R_T.shape == (3,3) and T_T.shape == (3,1), \
    f"{R_T.shape} and {T_T.shape}"
    final_T = np.identity(4)
    final_T[0:3,0:3] = R_T
    final_T[0:3,3:4] = T_T
    return final_T

def inv_T(T):
    assert T.shape == (4,4)
    inv_RT = np.transpose(T[0:3,0:3])
    inv_TT = -np.dot(inv_RT, T[0:3,3:4])
    inv_T = np.identity(4)
    inv_T[0:3,0:3] = inv_RT
    inv_T[0:3,3:4] = inv_TT
    return inv_T


In [17]:
#Test the robotic transformation ref:https://blog.csdn.net/chengde6896383/article/details/86738179
#T1 = np.array([[0.5, 0,0.866,3],[0.866,0,-0.5,2],[0.0,1,0,5],[0.0,0,0,1]]).reshape(4,4)
#print(T1)
#print(inv_T(T1))

[[ 0.5    0.     0.866  3.   ]
 [ 0.866  0.    -0.5    2.   ]
 [ 0.     1.     0.     5.   ]
 [ 0.     0.     0.     1.   ]]
[[ 0.5    0.866  0.    -3.232]
 [ 0.     0.     1.    -5.   ]
 [ 0.866 -0.5    0.    -1.598]
 [ 0.     0.     0.     1.   ]]


# Assignment 1

In [8]:
#Transformation from A to U: T1
#Transformation from A to B: T2
#Transformation from U to C: T3
#
T1 = np.array([[0.866, -0.5,0.0,11.0],[0.5,0.866,0.0,-1.0],[0.0,0.0,1.0,8.0],[0.0,0,0,1]])
T2 = np.array([[1.0,0.0,0.0,0.0],[0.0,0.866,-0.5,10.0],[0,0.5,0.866,-20.0],[0,0,0,1]])
T3 = np.array([[0.866,-0.5,0.0,-3.0],[0.433,0.750,-0.5,-3.0],[0.250,0.433,0.866,3.0],[0,0,0,1]])

print("T1= \n", T1, T1.shape)
print("T2= \n", T2, T2.shape)
print("T3= \n", T3, T3.shape)
print("####")

print("(T3·T1·T2^-1)^(-1) = ")
print(inv_T(np.dot(T3,np.dot(T1,inv_T(T2)))))
print("T2·T1^-1·T3^-1 = ")
print(np.dot(T2,np.dot(inv_T(T1),inv_T(T3))))

T1= 
 [[ 0.866 -0.5    0.    11.   ]
 [ 0.5    0.866  0.    -1.   ]
 [ 0.     0.     1.     8.   ]
 [ 0.     0.     0.     1.   ]] (4, 4)
T2= 
 [[  1.      0.      0.      0.   ]
 [  0.      0.866  -0.5    10.   ]
 [  0.      0.5     0.866 -20.   ]
 [  0.      0.      0.      1.   ]] (4, 4)
T3= 
 [[ 0.866 -0.5    0.    -3.   ]
 [ 0.433  0.75  -0.5   -3.   ]
 [ 0.25   0.433  0.866  3.   ]
 [ 0.     0.     0.     1.   ]] (4, 4)
####
(T3·T1·T2^-1)^(-1) = 
[[  0.5      0.75     0.433   -6.5744]
 [ -0.75     0.625   -0.2165  19.7868]
 [ -0.433   -0.2165   0.8749 -28.3161]
 [  0.       0.       0.       1.    ]]
T2·T1^-1·T3^-1 = 
[[  0.5      0.75     0.433   -6.5752]
 [ -0.75     0.625   -0.2165  19.7876]
 [ -0.433   -0.2165   0.8749 -28.3183]
 [  0.       0.       0.       1.    ]]


# Assignment 2

## Answer for Problem1

In [35]:
A_X,A_Y,A_Z = np.array([1,0,0]), np.array([0,1,0]), np.array([0,0,1])
B_X,B_Y,B_Z = np.array([-1,0,0]), np.array([0,-1,0]), np.array([0,0,1])
C_X, C_Y = np.array([0, 0, -1]), \
    np.array([-np.cos(np.deg2rad(60)),np.sin(np.deg2rad(60)),0])
C_Z = np.cross(C_X, C_Y)

# Obtain each of coordinate system
A = [A_X,A_Y,A_Z]
B = [B_X,B_Y,B_Z]
C = [C_X,C_Y,C_Z]

# Calculate Rotation and Translation matrix
RT_B2A,TT_B2A = calculate_R_transformation(B,A), np.array([3,0,0]).reshape(3,1)
RT_C2B,TT_C2B = calculate_R_transformation(C,B), np.array([0,0,2]).reshape(3,1)

final_T_B2A = add_T_transformation(RT_B2A,TT_B2A)
final_T_C2B = add_T_transformation(RT_C2B,TT_C2B)
print(f"{final_T_B2A}\n{final_T_C2B}")
#print(f"Here is the Latex version:\n{bmatrix(final_T_A2B)}\n{bmatrix(final_T_B2C)}")

[[-1.  0.  0.  3.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]
[[ 0.     0.5   -0.866  0.   ]
 [ 0.    -0.866 -0.5    0.   ]
 [-1.     0.     0.     2.   ]
 [ 0.     0.     0.     1.   ]]


In [29]:
#T_test = np.array([[0, 0.5,-0.866,0],[0,-0.866,-0.5,0],[-1,0,0,2],[0.0,0,0,1]]).reshape(4,4)
#print (inv_T(T_test))

In [36]:
final_T_C2A = np.dot(final_T_B2A, final_T_C2B)
final_T_A2C = inv_T(final_T_C2A)

In [37]:
print (f"2.27: The transformation from B to A:\n{final_T_B2A}")#\nfor Latex format Version\n{bmatrix(final_T_B2A)}")
print (f"2.28: The transformation from C to A:\n{final_T_C2A}")#\nfor Latex format Version\n{bmatrix(final_T_C2A)}")
print (f"2.29: The transformation from C to B:\n{final_T_C2B}")#\nfor Latex format Version\n{bmatrix(final_T_C2B)}")
print (f"2.30: The transformation from A to C:\n{final_T_A2C}")#\nfor Latex format Version\n{bmatrix(final_T_A2C)}")

2.27: The transformation from B to A:
[[-1.  0.  0.  3.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]
2.28: The transformation from C to A:
[[ 0.    -0.5    0.866  3.   ]
 [ 0.     0.866  0.5    0.   ]
 [-1.     0.     0.     2.   ]
 [ 0.     0.     0.     1.   ]]
2.29: The transformation from C to B:
[[ 0.     0.5   -0.866  0.   ]
 [ 0.    -0.866 -0.5    0.   ]
 [-1.     0.     0.     2.   ]
 [ 0.     0.     0.     1.   ]]
2.30: The transformation from A to C:
[[ 0.      0.     -1.      2.    ]
 [-0.5     0.866   0.      1.5   ]
 [ 0.866   0.5     0.     -2.5981]
 [ 0.      0.      0.      1.    ]]


## Answer for Problem2

In [45]:
A_X,A_Y,A_Z = np.array([1,0,0]), np.array([0,1,0]), np.array([0,0,1])
B_X,B_Y,B_Z = np.array([-1,0,0]), np.array([0,0,-1]), np.array([0,-1,0])
angle = 90 - 36.9
C_Y,C_Z = np.array([np.cos(np.deg2rad(angle)), -np.sin(np.deg2rad(angle)), 0]), \
np.array([0,0,-1])
C_X = np.cross(C_Y,C_Z)

A = [A_X,A_Y,A_Z]
B = [B_X,B_Y,B_Z]
C = [C_X,C_Y,C_Z]

RT_B2A,TT_B2A = calculate_R_transformation(B,A), np.array([0,4,2]).reshape(3,1)
RT_C2B,TT_C2B = calculate_R_transformation(C,B), np.array([3,0,0]).reshape(3,1)
final_T_B2A = add_T_transformation(RT_B2A,TT_B2A)
final_T_C2B = add_T_transformation(RT_C2B,TT_C2B)
print(f"{final_T_B2A}\n{final_T_C2B}")
# print(f"Here is the Latex version:\n{bmatrix(final_T_B2A)}\n{bmatrix(final_T_C2B)}")

[[-1.  0.  0.  0.]
 [ 0.  0. -1.  4.]
 [ 0. -1.  0.  2.]
 [ 0.  0.  0.  1.]]
[[-0.7997 -0.6004  0.      3.    ]
 [ 0.      0.      1.      0.    ]
 [-0.6004  0.7997  0.      0.    ]
 [ 0.      0.      0.      1.    ]]


In [46]:
final_T_C2A = np.dot(final_T_B2A, final_T_C2B)
final_T_A2C = inv_T(final_T_C2A)

In [47]:
print (f"2.31: The transformation from B to A:\n{final_T_B2A}")#\nfor Latex format Version\n{bmatrix(final_T_B2A)}")
print (f"2.32: The transformation from C to A:\n{final_T_C2A}")#\nfor Latex format Version\n{bmatrix(final_T_C2A)}")
print (f"2.33: The transformation from C to B:\n{final_T_C2B}")#\nfor Latex format Version\n{bmatrix(final_T_C2B)}")
print (f"2.34: The transformation from A to C:\n{final_T_A2C}")#\nfor Latex format Version\n{bmatrix(final_T_A2C)}")

2.31: The transformation from B to A:
[[-1.  0.  0.  0.]
 [ 0.  0. -1.  4.]
 [ 0. -1.  0.  2.]
 [ 0.  0.  0.  1.]]
2.32: The transformation from C to A:
[[ 0.7997  0.6004  0.     -3.    ]
 [ 0.6004 -0.7997  0.      4.    ]
 [ 0.      0.     -1.      2.    ]
 [ 0.      0.      0.      1.    ]]
2.33: The transformation from C to B:
[[-0.7997 -0.6004  0.      3.    ]
 [ 0.      0.      1.      0.    ]
 [-0.6004  0.7997  0.      0.    ]
 [ 0.      0.      0.      1.    ]]
2.34: The transformation from A to C:
[[ 0.7997  0.6004  0.     -0.0026]
 [ 0.6004 -0.7997  0.      5.    ]
 [ 0.      0.     -1.      2.    ]
 [ 0.      0.      0.      1.    ]]


# Assignment 3